# Chain
1. Simple
2. Sequential
3. Parallel
4. Conditional

1. Simple Chain

In [ ]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

prompt = PromptTemplate(
    template='Generate 5 interesting facts about {topic}',
    input_variables=['topic']
)

model = ChatOpenAI()

parser = StrOutputParser()

chain = prompt | model | parser

result = chain.invoke({'topic':'cricket'})

print(result)

chain.get_graph().print_ascii()

2. Sequential Chain

In [ ]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

prompt1 = PromptTemplate(
    template='Generate a detailed report on {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Generate a 5 pointer summary from the following text \n {text}',
    input_variables=['text']
)

model = ChatOpenAI()

parser = StrOutputParser()

chain = prompt1 | model | parser | prompt2 | model | parser

result = chain.invoke({'topic': 'Unemployment in India'})

print(result)

chain.get_graph().print_ascii()

3. Parallel Chain ( using langchain.schema.runnable.RunnableParallel(dict) )

Using 2 diff Chat Models

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel

load_dotenv()

model1 = ChatOpenAI()

model2 = ChatAnthropic(model_name='claude-3-haiku-20240307')

prompt1 = PromptTemplate(
    template='Generate short and simple notes from the following text \n {text}',
    input_variables=['text']
)

prompt2 = PromptTemplate(
    template='Generate 5 short question answers from the following text \n {text}',
    input_variables=['text']
)

prompt3 = PromptTemplate(
    template='Merge the provided notes and quiz into a single document \n notes -> {notes} and quiz -> {quiz}',
    input_variables=['notes', 'quiz']
)

parser = StrOutputParser()

parallel_chain = RunnableParallel({
    'notes': prompt1 | model1 | parser,     # these keys => labels/keys for respective output when invoked
    'quiz': prompt2 | model2 | parser
})

merge_chain = prompt3 | model1 | parser

chain = parallel_chain | merge_chain

text = """
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

The advantages of support vector machines are:

Effective in high dimensional spaces.

Still effective in cases where number of dimensions is greater than the number of samples.

Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.

Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of support vector machines include:

If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.

SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).

The support vector machines in scikit-learn support both dense (numpy.ndarray and convertible to that by numpy.asarray) and sparse (any scipy.sparse) sample vectors as input. However, to use an SVM to make predictions for sparse data, it must have been fit on such data. For optimal performance, use C-ordered numpy.ndarray (dense) or scipy.sparse.csr_matrix (sparse) with dtype=float64.
"""

result = chain.invoke({'text':text})

print(result)

chain.get_graph().print_ascii()


4. Conditional Chain  ( using langchain.schema.runnable.RunnableBranch(tuples) )

problem statement - 

classify sentiment of a user review, if

(a) +ve send to free hf model saying thanks

(b) -ve send to anthropic saying feedback received (later for automation can be used)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser

from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import Literal

load_dotenv()

model_openai = ChatOpenAI(model="gpt-4o-mini")
model_anthropic = ChatAnthropic(model_name='claude-3-haiku-20240307')
model_huggingface = ChatHuggingFace(llm=HuggingFaceEndpoint(repo_id="google/gemma-2-2b-it", task="text-generation"))

In [ ]:
review = 'high smartphon resell value, avg camera, next-gen processor'

# my code here

# model_openai.invoke(prompt_initial) # 'The sentiment of the review is mixed, but it leans towards positive due to the word "good" describing the smartphone, despite the mention of an "avg camera." Overall, I would classify the sentiment as **positive**.'
# here i want just 'positive' but llm is trash-taking. without this fixed i cant go to conditional chain

# fix using pydantic outputparser 
class ReviewSentiment(BaseModel):
    review_summary: str = Field(description='give the brief summary of the review')
    sentiment: Literal['positive', 'negative'] = Field(description='give the overall sentiment which fits best')

parser_for_sentiment = PydanticOutputParser(pydantic_object = ReviewSentiment)
str_parser = StrOutputParser()

template_initial_ip = PromptTemplate(template= 'Classify the sentiment of this review with positive or negative. Do not use words like "neutral" or "mixed". Be decisive' \
'\n {review} \n {format_instructions}', input_variables=['review'], partial_variables={'format_instructions': parser_for_sentiment.get_format_instructions()} )

# template_pos_ip = PromptTemplate(template='Write an appropriate response to this positive review \n {review}', input_variables=['review']) # OR even better -> using intermediate outputs
template_pos_ip = PromptTemplate(template='Write an appropriate response to this positive review \n {review_summary}', input_variables=['review_summary'])
template_neg_ip = PromptTemplate(template='Write an appropriate response to this negative review \n {review_summary}', input_variables=['review_summary'])

sentiment_chain = template_initial_ip | model_openai | parser_for_sentiment
# response = sentiment_chain.invoke(review) # review_summary='Good smartphone with an average camera.' sentiment='positive'
# print(response.sentiment) # target

# below is like 
# if condition1: chain1 
# elif condition2: chain2 
# else default_runnable
branch_chain = RunnableBranch( 
    (lambda x:x.sentiment == 'positive', template_pos_ip | model_huggingface | str_parser),
    (lambda x:x.sentiment == 'negative', template_neg_ip | model_anthropic   | str_parser),
    RunnableLambda(lambda x: "could not find sentiment")            )

chain = sentiment_chain | branch_chain


In [ ]:
review = 'high smartphon resell value, avg camera, next-gen processor'

print(chain.invoke({'review': review}))

chain.get_graph().print_ascii()